In [1]:
import os
import webbrowser
import matplotlib.pyplot as plt
import folium
import pandas as pd
import osmnx as ox
import networkx as nx
import numpy as np
from IPython.display import HTML, display
from project47.flp_data import *
from project47.data import *
from project47.routing import *
from project47.customer import Customer
from project47.flp_func import *


Status:  Optimal
Establish facility at site  18
Establish facility at site  19
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
26.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
20.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
66.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
8.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
25.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
2.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
4.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
46.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
14.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
2.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0

In [ ]:
# sol_fac_lat = [-43.5206429, -43.5262993]
# sol_fac_lon = [172.6566054, 172.5707267]

In [ ]:
# assigned_lat =[[-43.51256593, -43.51014213, -43.52071178, -43.56486542, -43.45819797, -43.50976918, -43.53743845, -43.4950214, -43.57484125, -43.52919052, -43.57152472, -43.513346999999996, -43.51458677, -43.55694703, -43.575494899999995, -43.56020022, -43.56914303, -43.5292406, -43.4991892, -43.47798017, -43.51719495, -43.56701963, -43.56272397, -43.50562883, -43.48958058, -43.55746693, -43.4598018, -43.47629057, -43.53505197, -43.56527187, -43.51580552, -43.57344848, -43.49970447, -43.56218525, -43.53017557, -43.55351712, -43.51156062, -43.575028, -43.55052053, -43.48246955, -43.518665000000006, -43.54864517, -43.54638645], [-43.51549592, -43.4862359, -43.52410697, -43.50605073, -43.5007566, -43.47809602, -43.51863488, -43.5752209, -43.48140632, -43.53185178, -43.55541497, -43.56613582, -43.54758475, -43.51893897, -43.540873, -43.54392452, -43.486431599999996, -43.4591649, -43.49745963, -43.47369873, -43.5335473, -43.51668153, -43.53306528, -43.55325085, -43.50804188, -43.55142697, -43.5329471, -43.5450706, -43.5251231]]
# assigned_lon =[[172.7052753, 172.6917437, 172.66926850000002, 172.64366730000003, 172.6304986, 172.7192564, 172.6946959, 172.685318, 172.6438364, 172.6368301, 172.7492679, 172.6771619, 172.6469783, 172.6986015, 172.7031687, 172.6744175, 172.6502418, 172.6680845, 172.6455667, 172.6492493, 172.61790990000003, 172.74092040000002, 172.716373, 172.7191128, 172.71655330000002, 172.6628107, 172.6585107, 172.6941083, 172.6612267, 172.7305007, 172.6588523, 172.75352980000002, 172.6618235, 172.6247526, 172.7350516, 172.7484264, 172.6302337, 172.761217, 172.63938969999998, 172.71775469999997, 172.689548, 172.6514904, 172.6798031], [172.5565999, 172.58429850000002, 172.53151430000003, 172.5911563, 172.5720542, 172.6010112, 172.5979173, 172.566302, 172.57670530000001, 172.5248438, 172.58958969999998, 172.5978911, 172.5326327, 172.57029409999998, 172.50878500000002, 172.58547869999998, 172.60158159999997, 172.61554109999997, 172.601709, 172.621265, 172.601977, 172.54083899999998, 172.55708959999998, 172.6108601, 172.60695969999998, 172.4883826, 172.5803813, 172.5548683, 172.5163788]]

In [ ]:
# lat = [-43.51256593, -43.51014213, -43.51549592, -43.52071178, -43.56486542, -43.45819797, -43.50976918, -43.4862359, -43.53743845, -43.52410697, -43.50605073, -43.5007566, -43.4950214, -43.47809602, -43.57484125, -43.52919052, -43.57152472, -43.513346999999996, -43.51458677, -43.51863488, -43.55694703, -43.5752209, -43.48140632, -43.575494899999995, -43.53185178, -43.55541497, -43.56020022, -43.56613582, -43.54758475, -43.56914303, -43.51893897, -43.540873, -43.5292406, -43.4991892, -43.47798017, -43.51719495, -43.54392452, -43.56701963, -43.56272397, -43.50562883, -43.48958058, -43.486431599999996, -43.4591649, -43.55746693, -43.4598018, -43.49745963, -43.47629057, -43.53505197, -43.56527187, -43.47369873, -43.5335473, -43.51580552, -43.57344848, -43.51668153, -43.49970447, -43.53306528, -43.56218525, -43.53017557, -43.55351712, -43.55325085, -43.51156062, -43.50804188, -43.575028, -43.55052053, -43.55142697, -43.5329471, -43.48246955, -43.518665000000006, -43.54864517, -43.5450706, -43.54638645, -43.5251231]
# lon = [172.7052753, 172.6917437, 172.5565999, 172.66926850000002, 172.64366730000003, 172.6304986, 172.7192564, 172.58429850000002, 172.6946959, 172.53151430000003, 172.5911563, 172.5720542, 172.685318, 172.6010112, 172.6438364, 172.6368301, 172.7492679, 172.6771619, 172.6469783, 172.5979173, 172.6986015, 172.566302, 172.57670530000001, 172.7031687, 172.5248438, 172.58958969999998, 172.6744175, 172.5978911, 172.5326327, 172.6502418, 172.57029409999998, 172.50878500000002, 172.6680845, 172.6455667, 172.6492493, 172.61790990000003, 172.58547869999998, 172.74092040000002, 172.716373, 172.7191128, 172.71655330000002, 172.60158159999997, 172.61554109999997, 172.6628107, 172.6585107, 172.601709, 172.6941083, 172.6612267, 172.7305007, 172.621265, 172.601977, 172.6588523, 172.75352980000002, 172.54083899999998, 172.6618235, 172.55708959999998, 172.6247526, 172.7350516, 172.7484264, 172.6108601, 172.6302337, 172.60695969999998, 172.761217, 172.63938969999998, 172.4883826, 172.5803813, 172.71775469999997, 172.689548, 172.6514904, 172.5548683, 172.6798031, 172.5163788]

In [ ]:
# fac_lat = [-43.46031, -43.3078924, -43.38570319999999, -43.4912018, -43.4904338, -43.5052942, -43.50629379999999, -43.5112185, -43.5318103, -43.54276340000001, -43.553157299999995, -43.533199700000004, -43.5398504, -43.5568711, -43.4599195, -43.475120000000004, -43.488491100000005, -43.517798299999995, -43.5206429, -43.5262993, -43.5387356]
# fac_lon = [172.62372, 172.5980646, 172.6576805, 172.6128727, 172.54666840000002, 172.6649538, 172.7286865, 172.5569844, 172.57370269999998, 172.52695580000002, 172.63605990000002, 172.6740007, 172.64272169999998, 172.7013435, 172.6223891, 172.65988000000002, 172.587223, 172.5883081, 172.6566054, 172.5707267, 172.6331705]

In [2]:
k = 2

coord = list(map(list, zip(lat, lon)))
fac_coord = list(map(list, zip(fac_lat, fac_lon)))

sol_fac_coord = list(map(list, zip( sol_fac_lat, sol_fac_lon)))
sol_fac_coord = [[-43.5206429, 172.6566054], [-43.5262993, 172.5707267]]
sol_fac_addr = ['19-23 Memorial Ave., Fendalton','288 Stanmore Rd, Richmond']
print(sol_fac_coord)

[[-43.5206429, 172.6566054], [-43.5262993, 172.5707267]]


In [3]:
coord_st = [-43.5320, 172.6397]
m = folium.Map(location = coord_st, zoom_start=11)
# for i in range(len(sol_fac_coord )):
#     popup = folium.Popup(sol_fac_addr[i], max_width=150, max_height = 40, color = "red")
#     folium.Marker(sol_fac_coord[i], popup="Optimal CP").add_to(m)
for i in range(len(lat)):
    folium.CircleMarker([lat[i],lon[i]], radius=100*weight[i], fill_opacity=0.7, color = "black").add_to(m)
for i in range(len(fac_lat)):
    folium.CircleMarker([fac_lat[i],fac_lon[i]], radius=0.5, fill_opacity=0.3, fill=True).add_to(m) #color="#007849"
  
# color='b',
m

#popup=('Suburb: ' + str(city).capitalize() + '<br>'
# 'Bike score: ' + str(bike) + '<br>'
                #  'Traffic level: ' + str(traffic) +'%')


In [4]:
coord_st = [-43.5320, 172.6397]
m = folium.Map(location = coord_st, zoom_start=11)
for i in range(len(sol_fac_coord )):
    popup = folium.Popup(sol_fac_addr[i], max_width=150, max_height = 40, color = "red")
    folium.Marker(sol_fac_coord[i], popup="Optimal CP").add_to(m)
for i in range(len(lat)):
    folium.CircleMarker([lat[i],lon[i]], radius=100*weight[i], fill_opacity=0.7, color = "black").add_to(m)
for i in range(len(assigned_lat)):
    for j in range(len(assigned_lat[i])):
        folium.PolyLine([(assigned_lat[i][j],assigned_lon[i][j]), (sol_fac_coord[i])], color="red", weight=0.5, opacity=1).add_to(m)

m


In [5]:

# direct to data folder
from numpy.random import Generator, PCG64
import random

cd = "/Users/karen.w/Desktop/project47_last_mile_logistics/data"
sample_data_csv = os.path.join(cd, "Toll_CHC_November_Sample_Data.csv")
# CHC_data_csv = os.path.join(cd, "christchurch_street.csv")
CHC_data_csv = "/Users/karen.w/Desktop/project47_last_mile_logistics/data/christchurch_street.csv"
sample_data_csv = "/Users/karen.w/Desktop/project47_last_mile_logistics/data/Toll_CHC_November_Sample_Data.csv"
sample_df, sample_sub_dict, CHC_df, CHC_df_grouped, CHC_sub_dict = read_data(
    sample_data_csv, CHC_data_csv)
seed = 123456789
rg = Generator(PCG64(seed))
latitude, longitude = get_sample(
    60,
    rg,
    cd,
    sample_df,
    sample_sub_dict,
    CHC_df_grouped,
    CHC_sub_dict,
    save=False,)
# cd = "/Users/karen.w/Desktop/project47_last_mile_logistics/data"

# get a random list of 20 numbers set at futile count threshld
rd = random.sample(range(0, len(latitude) - 1), 20)
futile_lat = []
futile_lon = []
for ind in rd:
    futile_lat.append(latitude[ind])
    futile_lon.append(longitude[ind])

assigned_lat = [[] for i in range(len(sol_fac_lat))]
assigned_lon = [[] for i in range(len(sol_fac_lat))]
# calculate the distance
for i in range(len(futile_lat)):
    lat_all = []
    lon_all = []
    # get the dist from the cusomter's house to the collection points
    lat_all = sol_fac_lat[:]
    lon_all = sol_fac_lon[:]
    lat_all.insert(0, futile_lat[i])
    lon_all.insert(0, futile_lon[i])
    coord_filename = None
    dist, tm = osrm_get_dist(
        cd,
        coord_filename,
        lat_all,
        lon_all,
        source=[0],
        save=False,
        host="localhost:5000",)
    # choose the closest collection point
    min_value = min(dist[0])
    # dist_threshold = 20000  # 20km
    # allow the package to be assigned to the closest collection point if the dist is within the threshold
    if min_value < 6000:
        min_ind = dist[0].index(min_value)
        # assign the package to its closest collection point
        assigned_lat[min_ind].append(futile_lat[i]) 
        assigned_lon[min_ind].append(futile_lon[i]) 
       
coord_st = [-43.5320, 172.6397]
m = folium.Map(location = coord_st, zoom_start=11)
# for i in range(len(sol_fac_coord )):
#     popup = folium.Popup(sol_fac_addr[i], max_width=150, max_height = 40, color = "red")
#     folium.Marker(sol_fac_coord[i], popup="Optimal CP").add_to(m)
for i in range(len(sol_fac_coord )):
    popup = folium.Popup(sol_fac_addr[i], max_width=150, max_height = 40, color = "red")
    folium.Marker(sol_fac_coord[i], popup="Optimal CP").add_to(m)
    folium.Circle(sol_fac_coord[i],radius=5000, weight = 1.2).add_to(m)
for i in range(len(latitude)):
    folium.CircleMarker([latitude[i], longitude[i]], radius=2.5, fill_opacity=1, fill=True , color = "black").add_to(m)
for i in range(len(futile_lat)):
    folium.CircleMarker([futile_lat[i], futile_lon[i]], radius=2.5, fill_opacity=1, fill=True, color = "red" ).add_to(m)
# for i in range(len(assigned_lat)):
#     for j in range(len(assigned_lat[i])):
#         folium.PolyLine([(assigned_lat[i][j],assigned_lon[i][j]), (sol_fac_coord[i])], color="red", weight=0.7, opacity=1).add_to(m)

# for i in range(len(futile_lat)):
#     folium.CircleMarker([futile_lat[i], futile_lon[i]], radius=2.5, fill_opacity=1, fill=True, color = "green" ).add_to(m)

# for i in range(len(fac_lat)):
#     folium.CircleMarker([fac_lat[i],fac_lon[i]], radius=0.5, fill_opacity=0.3, fill=True).add_to(m) #color="#007849"
  
# color='b',
m

In [17]:
# from folium.features import DivIcon

depo = [-43.5111688, 172.7319266]
# depo = [-43.5320, 172.6397]

alat = [-43.5111688, -43.52704282, -43.52451417, -43.51341075, -43.53609232, -43.50105713]
alon = [172.7319266, 172.6861508, 172.6314524, 172.57364980000003, 172.5287917, 172.7048862]
lat5, lon5 = get_sample(
    5,
    rg,
    cd,
    sample_df,
    sample_sub_dict,
    CHC_df_grouped,
    CHC_sub_dict,
    save=False,)
lat5.insert(0, depo[0])
lon5.insert(0, depo[1])
print(lat5)
print(lon5)
distances, tm = osrm_get_dist(
        cd,
        coord_filename,
        lat5,
        lon5,
        source=[],
        save=False,
        host="localhost:5000",)
locs = 6
depo = 0
# distances = np.array([[0, 2, 2], [2, 0, 4], [2, 4, 0]])
r = ORToolsRouting(locs, 1, depo)
dim, ind = r.add_dimension(distances, 0, 10, True, "distance")
r.routing.SetArcCostEvaluatorOfAllVehicles(ind)
s = r.solve()
route = s.routes[0]
route
r_lat = []
r_lon = []
for i in range(len(route)):
    r_lat.append(lat5[route[i]])
    r_lon.append(lon5[route[i]])
r_coord = list(map(list, zip(r_lat, r_lon)))
print(r_coord)

# coord_st = [-43.5320, 172.6397]
m = folium.Map(location = coord_st, zoom_start=11)
popup = folium.Popup([depo], max_width=150, max_height = 40, color = "red")
folium.Marker([-43.5112, 172.7320], popup="Depot").add_to(m)

for i in range(len(lat5)):
    folium.CircleMarker([lat5[i], lon5[i]], radius=2.5, fill_opacity=1, fill=True , color = "black").add_child(folium.Popup(str(i))).add_to(m)

folium.PolyLine(r_coord, color="red", weight=2, opacity=1).add_to(m)
m



[-43.5111688, -43.52704282, -43.52451417, -43.51341075, -43.53609232, -43.50105713]
[172.7319266, 172.6861508, 172.6314524, 172.57364980000003, 172.5287917, 172.7048862]
Solved: 1 (ROUTING_SUCCESS: Problem solved successfully.)
Objective: 0
[-43.5206429, 172.6566054]
